In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
# 设置随机种子以确保结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 检查是否有可用的GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")
# 自定义数据集类
class IntrusionDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.FloatTensor(labels)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# 定义GRU+LSTM
class GRU_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size_gru=128, hidden_size_lstm=64, dropout=0.3):
        super(GRU_LSTM, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size_gru, batch_first=True, bidirectional=True)
        self.dropout1 = nn.Dropout(dropout)
        self.lstm = nn.LSTM(hidden_size_gru*2, hidden_size_lstm, batch_first=True, bidirectional=True)
        self.dropout2 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size_lstm*2, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        gru_out, _ = self.gru(x)
        gru_out = self.dropout1(gru_out)
        
        lstm_out, _ = self.lstm(gru_out)
        lstm_out = self.dropout2(lstm_out[:, -1, :])
        
        out = self.relu(self.fc1(lstm_out))
        out = self.fc2(out)
        return out

# 数据加载
def load_data(train_self_path, train_nonself_path, test_self_path, test_nonself_path, unknown_path):
    train_nonself = pd.read_csv(train_nonself_path)
    train_self = pd.read_csv(train_self_path)
    train_self = train_self.sample(n=len(train_nonself),random_state=42)
    
    
    unknown = pd.read_csv(unknown_path)
    
    # 加载测试数据
    test_self = pd.read_csv(test_self_path)
    test_self = test_self.sample(n=5000,random_state=42)
    test_nonself = pd.read_csv(test_nonself_path)
    test_nonself = test_nonself.sample(n=5000,random_state=42)
    
    # 添加标签：自体为0，非自体为1
    train_self['label'] = 0
    train_nonself['label'] = 1
    test_self['label'] = 0
    test_nonself['label'] = 1
    
    # 合并训练集和测试集
    train_data = pd.concat([train_self, train_nonself], axis=0).reset_index(drop=True)
    train_data = train_data.sample(frac=1, random_state=42).reset_index(drop=True)
    test_data = pd.concat([test_self, test_nonself], axis=0).reset_index(drop=True)
    test_data = test_data.sample(frac=1, random_state=42).reset_index(drop=True)
    return train_data, test_data, unknown

# 数据预处理函数
def preprocess_data(train_data, test_data):
    # 分离特征和标签
    X_train = train_data.drop('label', axis=1).values
    y_train = train_data['label'].values
    X_test = test_data.drop('label', axis=1).values
    y_test = test_data['label'].values
    
    X_train_seq = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_seq = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    
    return X_train_seq, y_train, X_test_seq, y_test

# 训练模型函数
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=20, patience=5):
    model.to(device)
    
    # 用于早停的变量
    best_val_loss = float('inf')
    early_stop_counter = 0
    
    # 记录训练和验证损失
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        
        # 训练阶段
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            
            outputs = model(features)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss = train_loss / len(train_loader)
        train_losses.append(train_loss)
        
        # 验证阶段
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = criterion(outputs, labels.unsqueeze(1).float())
                val_loss += loss.item()
        
        val_loss = val_loss / len(val_loader)
        val_losses.append(val_loss)
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        # 更新学习率
        scheduler.step(val_loss)
        
        # 早停检查
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
            torch.save(model.state_dict(), 'best_gru_lstm_model.pth')
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f'Early stopping triggered after {epoch+1} epochs')
                break
    
    # 加载最佳模型
    model.load_state_dict(torch.load('best_gru_lstm_model.pth'))
    
    return model, train_losses, val_losses

# 评估未知覆盖率和误报率
def evaluate_unknown_coverage(model, unknown_data, threshold=0):
    # 预处理未知数据
    X_unknown = unknown_data.values
    X_unknown_seq = X_unknown.reshape(X_unknown.shape[0], 1, X_unknown.shape[1])
    unknown_dataset = torch.FloatTensor(X_unknown_seq)
    
    # 创建数据加载器
    unknown_loader = DataLoader(unknown_dataset, batch_size=64)
    
    model.eval()
    all_preds = []
    
    with torch.no_grad():
        for features in unknown_loader:
            features = features.to(device)
            outputs = model(features)
            predicted = (outputs >= threshold).float()
            all_preds.extend(predicted.cpu().numpy())
    
    all_preds = np.array(all_preds).flatten()
    
    # 计算未知覆盖率 - 被检测为异常的未知样本比例
    unknown_coverage = np.mean(all_preds)
    
    return unknown_coverage

# 评估误报率 - 在正常数据上
def evaluate_false_positive_rate(model, normal_data, threshold=0):
    # 预处理正常数据
    X_normal = normal_data.drop('label', axis=1).values
    X_normal_seq = X_normal.reshape(X_normal.shape[0], 1, X_normal.shape[1])
    
    normal_dataset = IntrusionDataset(X_normal_seq, np.zeros(len(X_normal)))
    normal_loader = DataLoader(normal_dataset, batch_size=64)
    
    model.eval()
    all_preds = []
    
    with torch.no_grad():
        for features, _ in normal_loader:
            features = features.to(device)
            outputs = model(features)
            predicted = (outputs >= threshold).float()
            all_preds.extend(predicted.cpu().numpy())
    
    all_preds = np.array(all_preds).flatten()
    
    # 计算误报率 - 正常样本被错误分类为异常的比例
    false_positive_rate = np.mean(all_preds)
    
    return false_positive_rate

# 评估模型性能
def evaluate_model(model, features, labels, threshold=0):
    model.eval()
    dataset = IntrusionDataset(features, labels)
    test_loader = DataLoader(dataset, batch_size=64)
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch_features, batch_labels in test_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            
            outputs = model(batch_features)
            predicted = (outputs >= threshold).float()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())
    
    all_preds = np.array(all_preds).flatten()
    all_labels = np.array(all_labels)
    
    # 计算评估指标
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    conf_matrix = confusion_matrix(all_labels, all_preds)
    
    return accuracy, precision, recall, f1, conf_matrix

# 计算最佳阈值
def find_optimal_threshold(model, X_val_seq, y_val):
    val_dataset = IntrusionDataset(X_val_seq, y_val)
    val_loader = DataLoader(val_dataset, batch_size=64)
    
    # 收集所有预测分数
    model.eval()
    all_scores = []
    all_labels = []
    
    with torch.no_grad():
        for features, labels in val_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            all_scores.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    all_scores = np.array(all_scores).flatten()
    all_labels = np.array(all_labels)
    
    # 尝试不同阈值
    thresholds = np.linspace(0.01, 0.99, 100)
    results = []
    
    for threshold in thresholds:
        # 计算验证集上的F1分数
        predicted = (all_scores >= threshold).astype(float)
        f1 = f1_score(all_labels, predicted)

        # 计算综合得分 (可以根据需要调整权重)
        score = f1
        results.append((threshold,f1,score))
    
    # 找到最佳阈值
    best_result = max(results, key=lambda x: x[1])
    return best_result


# 主函数
def main():
    unknown_types = ["dos", "probe", "r2l", "u2r"]
    for unknown_type in unknown_types:
 
        # 设置数据路径
        train_self_path = '../../check/self/train_self.csv'
        train_nonself_path = f'../../check/train/seed_{unknown_type}.csv'
        test_self_path = '../../check/self/test_self.csv'
        test_nonself_path = '../../check/nonself/test_nonself.csv'
        unknown_path = f'../../check/unknown/4type/{unknown_type}.csv'
        
        
        # 加载数据
        print("Loading data...")
        train_data, test_data, unknown = load_data(train_self_path, train_nonself_path, test_self_path, test_nonself_path,unknown_path)
        
        # 预处理数据
        print("Preprocessing data...")
        X_train, y_train, X_test, y_test = preprocess_data(train_data, test_data)
        
        # 将训练集分为训练集和验证集（80%训练，20%验证）
        train_size = int(0.8 * len(X_train))
        val_size = len(X_train) - train_size
        
        X_train_split, X_val = X_train[:train_size], X_train[train_size:]
        y_train_split, y_val = y_train[:train_size], y_train[train_size:]
        
        train_dataset = IntrusionDataset(X_train_split, y_train_split)
        val_dataset = IntrusionDataset(X_val, y_val)
        test_dataset = IntrusionDataset(X_test, y_test)
        batch_size = 64
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)
        
        input_size = X_train.shape[2]
        model = GRU_LSTM(input_size)
        
        # 权重
        pos_weight = torch.tensor([(y_train == 0).sum() / [(y_train == 1).sum()] ])
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(device))
        
        # 使用带权重衰减的优化器
        optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
        
        # 添加学习率调度器
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1)
        
        # 训练模型
        print("Training model...")
        start_time = time.time()
        model, train_losses, val_losses= train_model(
            model, 
            train_loader, 
            val_loader, 
            criterion, 
            optimizer,
            scheduler,
            num_epochs=50, 
            patience=10
        )
        training_time = time.time() - start_time
        print(f"Done training, time: {training_time:.2f} 秒")

        # 使用验证集找到最佳阈值
        print("在验证集上寻找最佳阈值...")
        best_result = find_optimal_threshold(model, X_val, y_val)
        best_threshold = best_result[0]
        
        # 使用测试集进行最终评估
        print("在测试集上进行最终评估...")
        test_accuracy, test_precision, test_recall, test_f1, test_conf_matrix = evaluate_model(
            model, X_test, y_test, threshold=best_threshold
        )

        # 在测试集上评估未知覆盖率和误报率
        test_unknown_coverage = evaluate_unknown_coverage(model, unknown, threshold=best_threshold)
        test_self_data = test_data[test_data['label'] == 0]
        test_fpr = evaluate_false_positive_rate(model, test_self_data, threshold=best_threshold)
        
        with open(f'{unknown_type}_results.txt', 'w') as f:
            f.write(f"Best Threshold: {best_threshold:.6f}\n")
            f.write("\nTest Set Results:\n")
            f.write(f"Accuracy: {test_accuracy:.4f}\n")
            f.write(f"Precision: {test_precision:.4f}\n")
            f.write(f"Recall: {test_recall:.4f}\n")
            f.write(f"F1 Score: {test_f1:.4f}\n")
            f.write(f"Unknown Coverage: {test_unknown_coverage:.4f}\n")
            f.write(f"False Positive Rate: {test_fpr:.4f}\n")
            f.write(f"Confusion Matrix: {test_conf_matrix}\n")
            
if __name__ == "__main__":
    main()

使用设备: cuda
Loading data...
Preprocessing data...


C:\Users\Administrator\AppData\Local\Temp\ipykernel_968\1689971584.py:325: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  pos_weight = torch.tensor([(y_train == 0).sum() / [(y_train == 1).sum()] ])


Training model...
Epoch 1, Train Loss: 0.6727, Val Loss: 0.6312
Epoch 2, Train Loss: 0.5135, Val Loss: 0.3465
Epoch 3, Train Loss: 0.2880, Val Loss: 0.2133
Epoch 4, Train Loss: 0.2301, Val Loss: 0.1905
Epoch 5, Train Loss: 0.2250, Val Loss: 0.1788
Epoch 6, Train Loss: 0.2102, Val Loss: 0.1688
Epoch 7, Train Loss: 0.1961, Val Loss: 0.1601
Epoch 8, Train Loss: 0.1964, Val Loss: 0.1532
Epoch 9, Train Loss: 0.1902, Val Loss: 0.1487
Epoch 10, Train Loss: 0.1851, Val Loss: 0.1461
Epoch 11, Train Loss: 0.1812, Val Loss: 0.1364
Epoch 12, Train Loss: 0.1721, Val Loss: 0.1419
Epoch 13, Train Loss: 0.1713, Val Loss: 0.1325
Epoch 14, Train Loss: 0.1616, Val Loss: 0.1307
Epoch 15, Train Loss: 0.1619, Val Loss: 0.1212
Epoch 16, Train Loss: 0.1541, Val Loss: 0.1202
Epoch 17, Train Loss: 0.1512, Val Loss: 0.1153
Epoch 18, Train Loss: 0.1415, Val Loss: 0.1097
Epoch 19, Train Loss: 0.1422, Val Loss: 0.1154
Epoch 20, Train Loss: 0.1426, Val Loss: 0.1026
Epoch 21, Train Loss: 0.1338, Val Loss: 0.1175
Epoc

C:\Users\Administrator\AppData\Local\Temp\ipykernel_968\1689971584.py:159: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_gru_lstm_mode

Done training, time: 10.16 秒
在验证集上寻找最佳阈值...
在测试集上进行最终评估...
Loading data...
Preprocessing data...
Training model...
Epoch 1, Train Loss: 0.5821, Val Loss: 0.3368
Epoch 2, Train Loss: 0.2927, Val Loss: 0.2269
Epoch 3, Train Loss: 0.1990, Val Loss: 0.1547
Epoch 4, Train Loss: 0.1504, Val Loss: 0.1437
Epoch 5, Train Loss: 0.1380, Val Loss: 0.1315
Epoch 6, Train Loss: 0.1280, Val Loss: 0.1238
Epoch 7, Train Loss: 0.1246, Val Loss: 0.1226
Epoch 8, Train Loss: 0.1232, Val Loss: 0.1159
Epoch 9, Train Loss: 0.1085, Val Loss: 0.1132
Epoch 10, Train Loss: 0.1185, Val Loss: 0.1077
Epoch 11, Train Loss: 0.1104, Val Loss: 0.1075
Epoch 12, Train Loss: 0.1080, Val Loss: 0.1064
Epoch 13, Train Loss: 0.1012, Val Loss: 0.0997
Epoch 14, Train Loss: 0.1024, Val Loss: 0.0997
Epoch 15, Train Loss: 0.0952, Val Loss: 0.0979
Epoch 16, Train Loss: 0.0964, Val Loss: 0.0969
Epoch 17, Train Loss: 0.0924, Val Loss: 0.1038
Epoch 18, Train Loss: 0.0940, Val Loss: 0.0969
Epoch 19, Train Loss: 0.0957, Val Loss: 0.0958
E

C:\Users\Administrator\AppData\Local\Temp\ipykernel_968\1689971584.py:159: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_gru_lstm_mode

在测试集上进行最终评估...
Loading data...
Preprocessing data...
Training model...
Epoch 1, Train Loss: 0.5484, Val Loss: 0.2787
Epoch 2, Train Loss: 0.2161, Val Loss: 0.1565
Epoch 3, Train Loss: 0.1698, Val Loss: 0.1400
Epoch 4, Train Loss: 0.1557, Val Loss: 0.1280
Epoch 5, Train Loss: 0.1504, Val Loss: 0.1231
Epoch 6, Train Loss: 0.1437, Val Loss: 0.1159
Epoch 7, Train Loss: 0.1392, Val Loss: 0.1182
Epoch 8, Train Loss: 0.1341, Val Loss: 0.1109
Epoch 9, Train Loss: 0.1306, Val Loss: 0.1149
Epoch 10, Train Loss: 0.1245, Val Loss: 0.1051
Epoch 11, Train Loss: 0.1247, Val Loss: 0.1066
Epoch 12, Train Loss: 0.1252, Val Loss: 0.1040
Epoch 13, Train Loss: 0.1155, Val Loss: 0.1124
Epoch 14, Train Loss: 0.1100, Val Loss: 0.0958
Epoch 15, Train Loss: 0.1070, Val Loss: 0.0947
Epoch 16, Train Loss: 0.1030, Val Loss: 0.0914
Epoch 17, Train Loss: 0.0977, Val Loss: 0.0899
Epoch 18, Train Loss: 0.0975, Val Loss: 0.0871
Epoch 19, Train Loss: 0.0879, Val Loss: 0.0819
Epoch 20, Train Loss: 0.0824, Val Loss: 0.080

C:\Users\Administrator\AppData\Local\Temp\ipykernel_968\1689971584.py:159: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_gru_lstm_mode

Loading data...
Preprocessing data...
Training model...
Epoch 1, Train Loss: 0.5391, Val Loss: 0.2740
Epoch 2, Train Loss: 0.2408, Val Loss: 0.1645
Epoch 3, Train Loss: 0.1989, Val Loss: 0.1524
Epoch 4, Train Loss: 0.1872, Val Loss: 0.1379
Epoch 5, Train Loss: 0.1803, Val Loss: 0.1362
Epoch 6, Train Loss: 0.1789, Val Loss: 0.1466
Epoch 7, Train Loss: 0.1731, Val Loss: 0.1301
Epoch 8, Train Loss: 0.1638, Val Loss: 0.1233
Epoch 9, Train Loss: 0.1577, Val Loss: 0.1152
Epoch 10, Train Loss: 0.1520, Val Loss: 0.1111
Epoch 11, Train Loss: 0.1490, Val Loss: 0.1041
Epoch 12, Train Loss: 0.1444, Val Loss: 0.1094
Epoch 13, Train Loss: 0.1380, Val Loss: 0.1002
Epoch 14, Train Loss: 0.1314, Val Loss: 0.0991
Epoch 15, Train Loss: 0.1243, Val Loss: 0.1072
Epoch 16, Train Loss: 0.1274, Val Loss: 0.1121
Epoch 17, Train Loss: 0.1237, Val Loss: 0.0941
Epoch 18, Train Loss: 0.1211, Val Loss: 0.1064
Epoch 19, Train Loss: 0.1160, Val Loss: 0.0890
Epoch 20, Train Loss: 0.1134, Val Loss: 0.0881
Epoch 21, Tra

C:\Users\Administrator\AppData\Local\Temp\ipykernel_968\1689971584.py:159: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_gru_lstm_mode